# Finite Capacity Region with Dropping vs M/M/1/K

This example shows that an FCR with dropping around a single queue behaves like an M/M/1/K queue, where K is the FCR capacity. Jobs arriving when the region is full are dropped (lost).

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*

In [ ]:
// Parameters
val arrivalRate = 0.8
val serviceRate = 1.0
val K = 3

In [ ]:
// Model 1: Queue with FCR (dropping)
val model1 = Network("FCR Dropping")

val source1 = Source(model1, "Source")
val queue1 = Queue(model1, "Queue", SchedStrategy.FCFS)
val sink1 = Sink(model1, "Sink")

val jobclass1 = OpenClass(model1, "Class1", 0)

source1.setArrival(jobclass1, Exp(arrivalRate))
queue1.setService(jobclass1, Exp(serviceRate))

val P1 = model1.initRoutingMatrix()
P1.set(jobclass1, jobclass1, source1, queue1, 1.0)
P1.set(jobclass1, jobclass1, queue1, sink1, 1.0)
model1.link(P1)

// Add FCR with dropping - jobs are lost when region is full
val fcr = model1.addRegion(listOf(queue1))
fcr.setGlobalMaxJobs(K)
fcr.setDropRule(jobclass1, true)  // true = drop jobs

In [ ]:
// Model 2: M/M/1/K using queue capacity
val model2 = Network("M/M/1/K")

val source2 = Source(model2, "Source")
val queue2 = Queue(model2, "Queue", SchedStrategy.FCFS)
queue2.setNumberOfServers(1)
queue2.setCapacity(K)  // Set queue capacity to K
val sink2 = Sink(model2, "Sink")

val jobclass2 = OpenClass(model2, "Class1", 0)

source2.setArrival(jobclass2, Exp(arrivalRate))
queue2.setService(jobclass2, Exp(serviceRate))

val P2 = model2.initRoutingMatrix()
P2.set(jobclass2, jobclass2, source2, queue2, 1.0)
P2.set(jobclass2, jobclass2, queue2, sink2, 1.0)
model2.link(P2)

In [ ]:
// Solve both models and compare results
val solver1 = JMT(model1, "seed", 23000, "samples", 100000)
val avgTable1 = solver1.avgTable
println("Model 1: FCR with Dropping")
avgTable1.print()

In [ ]:
val solver2 = JMT(model2, "seed", 23000, "samples", 100000)
val avgTable2 = solver2.avgTable
println("\nModel 2: M/M/1/K with Queue Capacity")
avgTable2.print()

## Explanation

This example demonstrates the equivalence between two modeling approaches:

1. **FCR with Dropping**: A queue inside a Finite Capacity Region with DROP policy (`setDropRule(true)`).

2. **M/M/1/K Queue**: A queue with explicit capacity set using `setCapacity(K)`.

Both models produce identical results because:
- When the FCR is full, arriving jobs are dropped
- When the M/M/1/K queue is full, arriving jobs are rejected
- Both behaviors are mathematically equivalent

This validates that FCRs correctly implement finite capacity constraints with job loss.